# **Equipe 01 - Motor classificador de tópicos**
---
Escola Politécnica da Universidade de Pernambuco - UPE/POLI

Disciplina: Inteligência Artificial/ 2020.2

Integrantes
- Matheus Phelipe (mpap@ecomp.poli.br)
- Murilo Stoldoni (mcs2@ecomp.poli.br)
- Nilton Vieira (nvs@ecomp.poli.br)
- Richard Jeremias (rjmr@ecomp.poli.br)


**Etapas envolvidas**

1. Recebimento da comunicação; ✅
2. Pré-processamento da pergunta; ✅
3. Geração do conjunto de atributos; ✅
4. Classificação da pergunta (assunto); ✅
5. Envio do resultado. ✅


**Instalação e imports iniciais**

In [ ]:
#Instalação dos pacotes necessários
!pip install seaborn
!pip install spacy
!pip install -U scikit-learn
!pip install delayed
!pip install -U nltk
!python -m spacy download pt


     |████████████████████████████████| 21.2 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


# **Imports**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import pickle
import pt_core_news_sm
import spacy
import re
import pandas as pd
import csv

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import RSLPStemmer
from spacy.lang.pt.stop_words import STOP_WORDS
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

pd.options.mode.chained_assignment = None
nltk.download('rslp') 
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Importação e tratamento dos dados**

In [ ]:
#IMPORTAÇÃO DA BASE DE DADOS 
filepath = 'BaseDados.csv'
data = pd.read_csv(filepath, sep=',')
print(len(data))
data_filter = data[(data['TIPO RESPOSTA'] != 'Acesso negado')]
print(len(data_filter))
#tamanho do processamento = Quantidade de duplas
tam_length = len(data_filter)

#NORMALIZANDO AS PALAVRAS
data_filter['pedido'] = data_filter['PEDIDO'].map(lambda x: x.lower())
coluna_tmp = [0] * tam_length

for i in range(tam_length):
  # nova coluna inserida no step anterior
  coluna_tmp[i] = data_filter.iloc[i]['pedido']
  letra_sem_acento = "a"
  coluna_tmp[i] = re.sub(r'([áàãâ])', letra_sem_acento, str(coluna_tmp[i]))
  letra_sem_acento = "e"
  coluna_tmp[i] = re.sub(r'([éê])', letra_sem_acento, str(coluna_tmp[i]))
  letra_sem_acento = "i"
  coluna_tmp[i] = re.sub(r'([í])', letra_sem_acento, str(coluna_tmp[i]))
  letra_sem_acento = "o"
  coluna_tmp[i] = re.sub(r'([óôõ])', letra_sem_acento, str(coluna_tmp[i]))
  letra_sem_acento = "u"
  coluna_tmp[i] = re.sub(r'([ú])', letra_sem_acento, str(coluna_tmp[i]))
  letra_sem_acento = "c"
  coluna_tmp[i] = re.sub(r'([ç])', letra_sem_acento, str(coluna_tmp[i]))

data_filter['pedido'] = coluna_tmp

# tirar caracteres especiais e os números
coluna_tmp = [0] * tam_length
for i in range(tam_length):
    # nova coluna inserida no step anterior
    coluna_tmp[i] = data_filter.iloc[i]['pedido']
    letra_sem_acento = ""
    coluna_tmp[i] = re.sub(r'([/\"-.,;:º@!?&%1234567890])',
                           letra_sem_acento, str(coluna_tmp[i]))
data_filter['pedido'] = coluna_tmp

# tirar palavras com tamanho menor 2
coluna_tmp = [0] * tam_length
for i in range(tam_length):
    tokens = str(data_filter.iloc[i]['pedido']).split()
    word_tmp = ""
    for word in tokens:
        if len(word) > 2:
            word_tmp = word_tmp + " " + word
    coluna_tmp[i] = word_tmp

data_filter['pedido'] = coluna_tmp

nlp = pt_core_news_sm.load()
stopwords = [k for k in STOP_WORDS]
new_stopwords = []
new_stopwords_list = stopwords.extend(new_stopwords)

coluna_tmp = [0] * tam_length
for i in range(tam_length):
    doc = nlp(str(data_filter.iloc[i]['pedido']))

    tokens = doc.text.split()

    doc_temp = ""
    for word in tokens:
        if (word in stopwords) == False:
            doc_temp = doc_temp + " " + word

    coluna_tmp[i] = doc_temp

data_filter['pedido'] = coluna_tmp
resultado = data_filter
resultado.to_csv('palavrasNormalizadas.csv')


3800
3591


In [ ]:
#stemmer
nlp = pt_core_news_sm.load()
stemmer = RSLPStemmer()

coluna_tmp = [0] * tam_length
for i in range(tam_length):
    doc = nlp(str(data_filter.iloc[i]['pedido']))

    tokens = doc.text.split()

    temp = ""
    for token in tokens:
        if token != "nan":
            temp = temp + " " + stemmer.stem(token)

    coluna_tmp[i] = temp.strip()
data_filter['pedido'] = coluna_tmp
print(data_filter['pedido'])
# insertColumn("stremer", coluna_tmp)
data_filter['palavra'] = data_filter['pedido'].map(lambda x: x.lower()) 
#data_filter['pedido'].map(lambda x: x.lower()) 

0       solicit ouvid seguint informaco relaca contrat...
1       relaca profes efet contrat are tecnolog inform...
2       relaca profes regim contrat temporari lingu po...
3       solicit relaca profes contrat municipi surubim...
4       solicit relaca profes educaca fisic contrat ge...
                              ...                        
3791    solicit shapefil bac hidrograf rio tejipi curs...
3794    bas lei estad art gost foss disponibil ato ren...
3795    solicit seguint informaco secret educaca perna...
3798    vist acess dad publ relacion process concessa ...
3799    tardesolicit informaco pge atribuico competenc...
Name: pedido, Length: 3591, dtype: object


**Criação das colunas com as stepwords**

In [ ]:
def saveStepWordsColumns(resultWithoutRepeat):
  nameFileColumns = "columns.csv"
  file = open(nameFileColumns, "w")
  writer = csv.writer(file)
  writer.writerow([item for item in resultWithoutRepeat])
  return nameFileColumns

resultadoSemRepeticao = set(data_filter['palavra'].str.cat(sep=',').replace(" ",",").split(","))
resultadoSemRepeticao = set(pd.read_csv(saveStepWordsColumns(resultadoSemRepeticao), sep=','))

### **Manipulação de número de amostras para treinamento**

In [ ]:
numeroAmostras = len(data_filter['ASSUNTO'])

In [ ]:
corpus = data_filter['palavra']
print(corpus)
def get_top_n_words(corpus, n=None, nWords = 3): #nWords define a quantidade de palavras armazenadas por perguntas. Por default, 3
    vec3 = CountVectorizer(ngram_range=(
        nWords, nWords), max_features=10000,  max_df=0.8).fit(corpus)
    bag_of_words = vec3.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx])
                  for word, idx in vec3.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n]

top3_words = get_top_n_words(corpus, n=numeroAmostras, nWords = 5)
top3_df = pd.DataFrame(top3_words)
top3_df.columns = ["Tri-gram", "Freq"]

resultado = top3_df
resultado.to_csv('topNWords_cv')

# plotar
def plotar(nomeX, nomeY, dados, filename='grafico.png'):
    sns.set(rc={'figure.figsize': (20, 15)})
    g = sns.barplot(y=nomeY, x=nomeX, data=dados)
    g.tick_params(labelsize=8)
    g.set_xlabel(nomeY, fontsize=10)
    g.set_ylabel(nomeX, fontsize=10)
    plt.savefig(filename)
    return

#plotar(nomeX="Freq", nomeY="Word", dados=top_df, filename='grafico1.png')
#plotar(nomeX="Freq", nomeY="Bi-gram", dados=top2_df, filename='grafico2.png')
#plotar(nomeX="Freq", nomeY="Tri-gram", dados=top3_df, filename='grafico3.png')


0       solicit ouvid seguint informaco relaca contrat...
1       relaca profes efet contrat are tecnolog inform...
2       relaca profes regim contrat temporari lingu po...
3       solicit relaca profes contrat municipi surubim...
4       solicit relaca profes educaca fisic contrat ge...
                              ...                        
3791    solicit shapefil bac hidrograf rio tejipi curs...
3794    bas lei estad art gost foss disponibil ato ren...
3795    solicit seguint informaco secret educaca perna...
3798    vist acess dad publ relacion process concessa ...
3799    tardesolicit informaco pge atribuico competenc...
Name: palavra, Length: 3591, dtype: object


# **Descobrindo o número de vezes que o mesmo assunto aparece**

In [ ]:
listaAssunto = [x.lower() for x in set(data_filter['ASSUNTO'])]
dicionario = {'pedido' :  data_filter['pedido'], 'assunto' : [x.lower() for x in data_filter['ASSUNTO']]} #lower nos assuntos

listaOutros = set() 

def lowOccurrenceTopics(totalTopics, rate = 1): #método detecta baixa ocorrência de tópicos inferiores a taxa 'rate'
  othersSet = set()
  for key, tamanho in totalTopics.items():
    if ((tamanho/len(data_filter))*100) < rate: #tamanho de inputs de data_filter agora pode ser variável
      othersSet.add(key)
  return othersSet

def updateDataSetLabels(finalTopicList, topicsList, labelName = "outros"): #método responsável por aplicar as mudanças nos rótulos no dicionáro composto pela chave 'pedido' e 'assunto'
  count = 0
  for item in finalTopicList:
    if(item in topicsList):
      finalTopicList[count] = labelName
    count = count + 1
  return finalTopicList

listaOutros = lowOccurrenceTopics(dict(Counter(dicionario['assunto'])), rate = 8)
#print(listaOutros)
#atualização de rótulos feita direto em dicionário que contém pedido -> assunto
#print(dicionario['assunto'])
dicionario['assunto'] = updateDataSetLabels(dicionario['assunto'], listaOutros, 'outros')

#print(dicionario['assunto'])

#UTILIZAR 
#listaAssunto = dict(Counter(listaAssunto)) #obtenção de ocorrência de assuntos
#listaAssunto = {k: v for k, v in sorted(listaAssunto.items(), key=lambda item: item[1])} #ordenamento crescente por quantidade
#print(listaAssunto)

## **Balancear assuntos**
Deixar o número de corrências dos assuntos o mais próximo possível, para não haver uma descrepância de alguns assuntos com muitas ocorrências e outros com poucas ocorrências.

In [ ]:
print(dict(Counter(dicionario['assunto'])))

#Salvar assuntos
def saveTopics(topics):
  file = open('topics.csv', "w")
  writer = csv.writer(file)
  print(set(topics))
  writer.writerow([item for item in set(topics)])

saveTopics(dicionario['assunto'])



{'servidores': 860, 'outros': 1510, 'execução orçamentária e financeira': 379, 'licitações e contratos': 429, 'segurança pública': 413}
{'execução orçamentária e financeira', 'outros', 'servidores', 'segurança pública', 'licitações e contratos'}


# **Binarização das stepwords**

In [ ]:
nameFileColumns = "columns.csv"
file = open(nameFileColumns, "w")
writer = csv.writer(file)

data = list(resultado['Tri-gram'])
last_data = [item.split() for item in data]

listaAssuntos = dicionario['assunto']

listaTotal = []
listaTeste = []
index = 0
for n_palavras_perguntas in last_data:
  listaParcial = []
  for cada_coluna in resultadoSemRepeticao:
    if cada_coluna in n_palavras_perguntas:
      listaParcial.append(1) 
    elif cada_coluna is None:
      listaParcial.append(None)
    else:
      listaParcial.append(0)
  if index < int(numeroAmostras*0.8):
    listaTotal.append(listaParcial)
  else:
    listaTeste.append(listaParcial)
  index = index + 1

for x in listaTotal:
  writer.writerow([item for item in x])

# **Divisão em dados de testes e treinamento**

In [ ]:
#divisão dos dados em 20% para testes e 80% para treinamento
x, y = None, None
x = coluna_tmp
X_train, X_test, y_train, y_test = None, None, None, None

Pré-processamento de input do usuário

In [ ]:
def proccessText(text): #chamar este método para pré-processar pergunta
  text = text.lower()
  text = re.sub(r'([áàãâ])', 'a', str(text))
  text = re.sub(r'([éê])', 'e', str(text))
  text = re.sub(r'([í])', 'i', str(text))
  text = re.sub(r'([óôõ])', 'o', str(text))
  text = re.sub(r'([ú])', 'u', str(text))
  text = re.sub(r'([ç])', 'c', str(text))
  # tirar caracteres especiais e os números
  text = re.sub(r'([/\"-.,;:º@!?&%1234567890])',"", str(text))
  
  # tirar palavras com tamanho menor 2
  coluna_tmp = [0] * len(text)
  tokens = str(text).split()
  word_tmp = ""
  for word in tokens:
    if len(word) > 3:
      word_tmp = word_tmp + " " + word
  text = word_tmp

  nlp = pt_core_news_sm.load()
  stopwords = [k for k in STOP_WORDS]
  new_stopwords = []
  new_stopwords_list = stopwords.extend(new_stopwords)
  doc = nlp(str(text))
  tokens = doc.text.split()

  doc_temp = ""
  for word in tokens:
    if (word in stopwords) == False:
      doc_temp = doc_temp + " " + word
  text = doc_temp
  return pergunta(text)


def binarizeInput(text, wordList): #como binarizar input???
  binaryList = []
  for word in wordList:
    if(word in text):
      binaryList.append(1)
    else:
      binaryList.append(0)
  return binaryList

def feedback():
  
  pass

def pergunta(text):
  text = proccessText(text)
  doc = nlp(str(text))
  tokens = doc.text.split()
  temp = ""
  for token in tokens:
    if token != "nan":
      temp = temp + " " + stemmer.stem(token)
  return temp

#print(proccessText("olá eu não gostaria matar criança feminicídio solicito de saber essa informação"))
#nojeira = "olá eu não gostaria matar criança feminicídio solicito de saber essa informação"
#aux = pergunta(nojeira)
#binary = binarizeInput(aux, resultadoSemRepeticao)
#binary = pergunta(proccessText("olá eu não gostaria matar criança feminicídio solicito de saber essa informação"), resultadoSemRepeticao)
#print(binary)

# **Treinamento e predição com dados de testes**

In [ ]:
!pip install imbalanced-learn

In [ ]:
#uso de random forest algorithm pra treinar modelo
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree
#from imblearn.under_sampling import RandomUnderSampler 

def train():
  X, y = listaTotal,listaAssuntos[:int(numeroAmostras*0.8)]
  #clf = tree.DecisionTreeClassifier() #32%
  #clf = clf.fit(X, y)
  #tree.plot_tree(clf)

  #clf = RandomForestClassifier(max_depth=10, random_state=0)
  #clf.fit(X, y)
  # usar técnica under-sampling
  clf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=5, bootstrap = True,  random_state=0) #50%
  #rus = RandomUnderSampler()
  #X_res, y_res = rus.fit_sample(X, y)
  #clf.fit(X_res, y_res)
  clf.fit(X, y)

  #clf = ExtraTreesClassifier(n_estimators=100, random_state=0) #34%
  #clf.fit(X, y)
  return clf

clf = train()


# **Status do modelo treinado**

In [ ]:
y_pred = []
X_teste,y_teste = listaTeste,listaAssuntos[(int(numeroAmostras*0.8)):numeroAmostras]  
print(clf.score(X_teste,y_teste))
for index in range(len(y_teste)):
  value = clf.predict([X_teste[index]])
  y_pred.append(value)

#Salvar assuntos
def saveResults(x, y): #passar lista de perguntas antes de binarizar....
  file = open('results.csv', "w")
  writer = csv.writer(file)
  if(len(x)==len(y)): #deve existir o par (x,y)
    with open('results.txt', 'w') as f:
      f.write('PEDIDO   ASSUNTO\n')
      for index in range(len(x)):
        f.write(f'{x[index]}      {y[index]};\n')
    
    
saveResults(['1', '2', '3'] ,['4', '5', '6']) #exemplo

0.5048678720445062


# **Interface de comunicação**

In [ ]:
!pip install Flask

In [ ]:
from flask import Flask, jsonify

#criada instancia da classe
app = Flask(__name__)

#-----------INTERAÇÃO COM O FRONT
#recebimento da comunicação
@app.route("/", methods = ['POST'])
def getInfo():
  #inicio do processamento,
  #aqui deve ser enviado o resultado quando estiver disponível
  #criar método predict
  #avaliar json recebido...
  return jsonify(
      tema = "tema x"
  )


#classificar novamente se necessário
@app.route("/send_feedback", methods = ['POST'])
def getFeedback():
  return "feedback received..."

#envio dos assuntos para equipe 4 e 5, retorna lista de assuntos processada
@app.route("/get_topics", methods = ['GET'])
def getTopics():
  topics = list(pd.read_csv('topics.csv'))
  return jsonify(topics)


#-----------INTERAÇÃO COM AS DEMAIS ENGINES
app.run(port=5100) #aplicação disponível na porta: 5100, ip: localhost

**Chaves JSON**


descricao_tratada, resposta_publica, unidade_gestora, assunto